<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/examples/blob/main/spark-ml/linearregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#adapted from https://towardsdatascience.com/apache-spark-mllib-tutorial-ec6f1cb336a9 

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#install spark. we are using the one that uses hadoop as the underlying scheduler.
!wget -q https://downloads.apache.org/spark//spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!ls -l
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

total 293932
drwxr-xr-x  1 root root      4096 Apr 25 13:46 sample_data
drwxr-xr-x 13  501 1000      4096 Jan 20 20:10 spark-3.2.1-bin-hadoop3.2
-rw-r--r--  1 root root 300971569 Jan 20 21:37 spark-3.2.1-bin-hadoop3.2.tgz


In [4]:
# w use the boston housing dataset
!wget https://raw.githubusercontent.com/vu-topics-in-big-data-2022/examples/main/spark-ml/boston_housing.csv

--2022-04-28 15:26:49--  https://raw.githubusercontent.com/vu-topics-in-big-data-2022/examples/main/spark-ml/boston_housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/plain]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.003s  

2022-04-28 15:26:49 (10.2 MB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [5]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)
data.show(3)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 3 rows



In [6]:
#its good to check the spark explain
data.explain()

== Physical Plan ==
FileScan csv [crim#16,zn#17,indus#18,chas#19,nox#20,rm#21,age#22,dis#23,rad#24,tax#25,ptratio#26,b#27,lstat#28,medv#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/boston_housing.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<crim:double,zn:double,indus:double,chas:int,nox:double,rm:double,age:double,dis:double,rad...




In [7]:
#Spark ML’s algorithms expect the data to be represented in two columns: Features and Labels.
#Features is an array of data points of all the features to be used for prediction. Labels contain the output label for each data point.
# Use Vector Assembler to create features
feature_columns = data.columns[:-1]
from pyspark.ml.feature import VectorAssembler
help(VectorAssembler)

Help on class VectorAssembler in module pyspark.ml.feature:

class VectorAssembler(pyspark.ml.wrapper.JavaTransformer, pyspark.ml.param.shared.HasInputCols, pyspark.ml.param.shared.HasOutputCol, pyspark.ml.param.shared.HasHandleInvalid, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  VectorAssembler(*, inputCols=None, outputCol=None, handleInvalid='error')
 |  
 |  A feature transformer that merges multiple columns into a vector column.
 |  
 |  .. versionadded:: 1.4.0
 |  
 |  Examples
 |  --------
 |  >>> df = spark.createDataFrame([(1, 0, 3)], ["a", "b", "c"])
 |  >>> vecAssembler = VectorAssembler(outputCol="features")
 |  >>> vecAssembler.setInputCols(["a", "b", "c"])
 |  VectorAssembler...
 |  >>> vecAssembler.transform(df).head().features
 |  DenseVector([1.0, 0.0, 3.0])
 |  >>> vecAssembler.setParams(outputCol="freqs").transform(df).head().freqs
 |  DenseVector([1.0, 0.0, 3.0])
 |  >>> params = {vecAssembler.inputCols: ["b", "a"], vecAssembler.outputCol: "ve

In [8]:
#VectorAssembler is a transformer that combines a given list of columns into a single vector column. 
#It is useful for combining raw features and features generated by different feature transformers into a single feature vector,
# in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. 
# In each row, the values of the input columns will be concatenated into a vector in the specified order.

# Note this is one of the examples of transformers available to work with features
# https://spark.apache.org/docs/3.1.1/ml-features.html

assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [9]:
data_2 = assembler.transform(data)

In [10]:
data_2.show(3, truncate=False) #notice the new column features

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-------------------------------------------------------------------------+
|crim   |zn  |indus|chas|nox  |rm   |age |dis   |rad|tax|ptratio|b     |lstat|medv|features                                                                 |
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-------------------------------------------------------------------------+
|0.00632|18.0|2.31 |0   |0.538|6.575|65.2|4.09  |1  |296|15.3   |396.9 |4.98 |24.0|[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98]  |
|0.02731|0.0 |7.07 |0   |0.469|6.421|78.9|4.9671|2  |242|17.8   |396.9 |9.14 |21.6|[0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14] |
|0.02729|0.0 |7.07 |0   |0.469|7.185|61.1|4.9671|2  |242|17.8   |392.83|4.03 |34.7|[0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03]|
+-------+----+-----+----+-----+-----+----+------+---

In [11]:
#lets split the data
train, test = data_2.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml.regression import LinearRegression

In [13]:
algo = LinearRegression(featuresCol="features", labelCol="medv") # label column is the predicted variable
 

In [14]:
model = algo.fit(train) #thats it

In [15]:
#evaluation
evaluation_summary = model.evaluate(test)
print(evaluation_summary.meanAbsoluteError,evaluation_summary.rootMeanSquaredError,evaluation_summary.r2)

3.6632112798010645 5.18922296259853 0.7172646079185679


In [16]:
#lets use the model 
predictions = model.transform(test)
predictions.explain()

== Physical Plan ==
*(1) Project [crim#16, zn#17, indus#18, chas#19, nox#20, rm#21, age#22, dis#23, rad#24, tax#25, ptratio#26, b#27, lstat#28, medv#29, features#119, UDF(features#119) AS prediction#342]
+- *(1) Sample 0.7, 1.0, false, 394076289428001447
   +- *(1) Sort [crim#16 ASC NULLS FIRST, zn#17 ASC NULLS FIRST, indus#18 ASC NULLS FIRST, chas#19 ASC NULLS FIRST, nox#20 ASC NULLS FIRST, rm#21 ASC NULLS FIRST, age#22 ASC NULLS FIRST, dis#23 ASC NULLS FIRST, rad#24 ASC NULLS FIRST, tax#25 ASC NULLS FIRST, ptratio#26 ASC NULLS FIRST, b#27 ASC NULLS FIRST, lstat#28 ASC NULLS FIRST, medv#29 ASC NULLS FIRST, features#119 ASC NULLS FIRST], false, 0
      +- *(1) Project [crim#16, zn#17, indus#18, chas#19, nox#20, rm#21, age#22, dis#23, rad#24, tax#25, ptratio#26, b#27, lstat#28, medv#29, UDF(struct(crim, crim#16, zn, zn#17, indus, indus#18, chas_double_VectorAssembler_c7743ec70794, cast(chas#19 as double), nox, nox#20, rm, rm#21, age, age#22, dis, dis#23, rad_double_VectorAssembler_c7743

In [17]:
predictions.show(5)

+-------+-----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|   zn|indus|chas|   nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+-----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096| 55.0| 2.25|   0| 0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...| 27.61678646477938|
|0.01432|100.0| 1.32|   0| 0.411|6.816|40.5|8.3248|  5|256|   15.1| 392.9| 3.95|31.6|[0.01432,100.0,1....|33.157412936460474|
|0.01439| 60.0| 2.93|   0| 0.401|6.604|18.8|6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...|32.281357633087566|
|0.01951| 17.5| 1.38|   0|0.4161|7.104|59.5|9.2229|  3|216|   18.6|393.24| 8.05|33.0|[0.01951,17.5,1.3...|21.426546716515848|
|0.02055| 85.0| 0.74|   0|  0.41|6.383|35.7|9.1876|  2|313|   17.3| 396.9| 5.77|24.7|[0.02055,85.0,0.7...| 25.20379536